## Reading of ASCII files created for cams diagnostics tool

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import os
from helper_funcs import read_file_custom

data_dir = "./data/michael_ascii_read/"
case_ok = "./data/from_ada/table_GLBL_ANN_obs_FIXED.asc"

VERBOSE = True

In [2]:
files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".webarchive")]

for file in files:
    print(file)
    
test_file = files[0]

print("TEST FILE: {}".format(os.path.basename(test_file)))

./data/michael_ascii_read/N1850_f19_tn14_r227_ctrl (yrs 310-340).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r227_ctrl (yrs 80-110).webarchive
./data/michael_ascii_read/N1850_f09_tn14_230218 (yrs 1-20).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r227_ctrl (yrs 185-215).webarchive
./data/michael_ascii_read/N1850C53CLM45L32_f09_tn11_191017 (yrs 71-100).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r265_ctrl_20180411 (yrs 90-120).webarchive
TEST FILE: N1850_f19_tn14_r227_ctrl (yrs 310-340).webarchive


Try read first file as is with pandas

In [3]:
try:
    frame = pd.read_csv(test_file, encoding="latin-1")
except Exception as e:
    print(repr(e))
frame.head()

,bplist00Ñ_WebMainResourceÕ
0,_WebResourceTextEncodingName^WebResourceUR...
1,TEST CASE: N1850_f19_tn14_r227_ctrl (yrs 310-340)
2,CONTROL CASE: OBS data
3,Variable N1850_f19_tn14_r227_ctrl OBS...
4,...


In [4]:
frame = pd.read_csv(case_ok)
frame.head()
frame.shape

(67, 1)

This did not really work since the data is not splitted by columns but includes one column containing the content of each row. The reading has to be done from scratch, especially also because there is some variables with too long names (e.g. L.25 and L. 28) that stick together the first two columns. 

This folder contains a file ``helper_funcs.py`` in which I defined a custom read function ``read_file_custom`` that can convert these files into pandas dataframes.

In [5]:
help(read_file_custom)

Help on function read_file_custom in module helper_funcs:

read_file_custom(fpath, verbose=False)
    Custom ASCII conversion method 
    
    Parameters
    ----------
    fpath : str
        path to file location
    verbose : bool
        if True, print output (defaults to False)
    Returns
    -------
    Dataframe 
        pandas data frame ready for further analysis



Now put the results into dataframe.

In [6]:
df = read_file_custom(test_file, VERBOSE)
df.head()

Ignoring line: bplist00Ñ_WebMainResourceÕ	
Ignoring line: 
Ignoring line: 
Ignoring line: _WebResourceTextEncodingName^WebResourceURL_WebResourceFrameName_WebResourceData_WebResourceMIMETypeUUTF-8_}http://ns2345k.web.sigma2.no/noresm_diagnostics/N1850_f19_tn14_r227_ctrl/CAM_DIAG/yrs310to340-obs/set1/table_GLBL_ANN_obs.ascPOf<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">DIAG SET 1: ANN MEANS GLOBAL
Ignoring line:  
Ignoring line:  
Ignoring line:  
Problem case FSNTOA_CERES-EBAF
Problem case FSNTOAC_CERES-EBAF
Test case: N1850_f19_tn14_r227_ctrl 
Control case: OBS data


,,,Model,Obs,Bias,RMSE
Variable,Run,Years,,,,
RESTOM,N1850_f19_tn14_r227_ctrl,310-340,0.327,NaN,NaN,NaN
RESSURF,N1850_f19_tn14_r227_ctrl,310-340,0.337,NaN,NaN,NaN
RESTOA_CERES-EBAF,N1850_f19_tn14_r227_ctrl,310-340,2.412,0.992,1.420,8.937
RESTOA_ERBE,N1850_f19_tn14_r227_ctrl,310-340,2.412,0.059,2.353,8.906
SOLIN_CERES-EBAF,N1850_f19_tn14_r227_ctrl,310-340,340.200,340.054,0.146,0.417


See if the corrected file also works.

In [7]:
df1 = read_file_custom(case_ok, VERBOSE)

Ignoring line: DIAG SET 1: ANN MEANS GLOBAL
Ignoring line:  
Ignoring line:  
Ignoring line:  
Problem case FSNTOA_CERES-EBAF
Problem case FSNTOAC_CERES-EBAF
Test case: N1850_f19_tn14_r265_ctrl_20180411 
Control case: OBS data


Reading worked, check, if both dataframes have the same dimension.

In [8]:
print(df.shape)
print(df1.shape)

(63, 4)
(63, 4)


In [46]:
dfs = []
#only load first two for now
for k in range(2):
    df = read_file_custom(files[k], verbose=False)
    #print(df.head(), "\n\n")
    dfs.append(df)
    if VERBOSE:
        print("Number of variables / values")
        print(df.shape)        

Number of variables / values
(63, 4)
Number of variables / values
(63, 4)


In [47]:
merged = pd.concat(dfs)
merged

,,,Model,Obs,Bias,RMSE
Variable,Run,Years,,,,
RESTOM,N1850_f19_tn14_r227_ctrl,310-340,0.327,NaN,NaN,NaN
RESSURF,N1850_f19_tn14_r227_ctrl,310-340,0.337,NaN,NaN,NaN
RESTOA_CERES-EBAF,N1850_f19_tn14_r227_ctrl,310-340,2.412,0.992,1.420,8.937
RESTOA_ERBE,N1850_f19_tn14_r227_ctrl,310-340,2.412,0.059,2.353,8.906
SOLIN_CERES-EBAF,N1850_f19_tn14_r227_ctrl,310-340,340.200,340.054,0.146,0.417
SOLIN_CERES,N1850_f19_tn14_r227_ctrl,310-340,340.200,341.479,-1.279,1.296
CLDTOT_ISCCP,N1850_f19_tn14_r227_ctrl,310-340,68.234,66.800,1.435,13.078
CLDTOT_CLOUDSAT,N1850_f19_tn14_r227_ctrl,310-340,68.234,66.824,1.411,10.952
FLDS_ISCCP,N1850_f19_tn14_r227_ctrl,310-340,354.846,343.347,11.499,17.664


In [48]:
bias_mean = merged.groupby(["Variable", "Run", "Years", "Bias"]).mean()
print(bias_mean.shape)
bias_mean

(114, 3)


Model  \
Variable           Run                       Years   Bias               
CLDTOT_CLOUDSAT    N1850_f19_tn14_r227_ctrl  310-340  1.411    68.234   
                                             80-110   2.133    68.956   
CLDTOT_ISCCP       N1850_f19_tn14_r227_ctrl  310-340  1.435    68.234   
                                             80-110   2.157    68.956   
FLDS_ISCCP         N1850_f19_tn14_r227_ctrl  310-340  11.499  354.846   
                                             80-110   5.162   348.508   
FLNS_ISCCP         N1850_f19_tn14_r227_ctrl  310-340  6.847    56.272   
                                             80-110   7.167    56.592   
FLUTC_CERES        N1850_f19_tn14_r227_ctrl  310-340  0.599   267.477   
                                             80-110  -1.614   265.265   
FLUTC_CERES-EBAF   N1850_f19_tn14_r227_ctrl  310-340  1.426   267.477   
                                             80-110  -0.786   265.265   
FLUTC_ERBE         N1850_f19_tn14_r227_ctrl  310-340  3.048   267.477   
                                             80-110   0.836   265.265   
FLUT_CERES         N1850_f19_tn14_r227_ctrl  310-340  3.539   242.502   
                                             80-110   1.272   240.235   
FLUT_CERES-EBAF    N1850_f19_tn14_r227_ctrl  310-340  2.928   242.502   
                                             80-110   0.660   240.235   
FLUT_ERBE          N1850_f19_tn14_r227_ctrl  310-340  8.556   242.502   
                                             80-110   6.289   240.235   
FSDS_ISCCP         N1850_f19_tn14_r227_ctrl  310-340  1.228   190.618   
                                             80-110   1.216   190.606   
FSNS_ISCCP         N1850_f19_tn14_r227_ctrl  310-340  1.365   167.258   
                                             80-110  -0.017   165.875   
FSNS_LARYEA        N1850_f19_tn14_r227_ctrl  310-340  13.169  167.258   
                                             80-110   11.787  165.875   
FSNTOAC_CERES      N1850_f19_tn14_r227_ctrl  310-340 -4.183   290.519   
                                             80-110  -5.747   288.956   
FSNTOAC_CERES-EBAF N1850_f19_tn14_r227_ctrl  310-340  2.882   290.519   
                                             80-110   1.318   288.956   
...                                                               ...   
SST_HADISST_PI     N1850_f19_tn14_r227_ctrl  310-340  2.385    22.753   
                                             80-110   1.756    22.232   
STRESS_MAG_ERS     N1850_f19_tn14_r227_ctrl  310-340  0.000     0.065   
                                             80-110   0.001     0.065   
STRESS_MAG_JRA25   N1850_f19_tn14_r227_ctrl  310-340 -0.016     0.065   
                                             80-110  -0.015     0.065   
STRESS_MAG_LARYEA  N1850_f19_tn14_r227_ctrl  310-340 -0.007     0.065   
                                             80-110  -0.006     0.065   
SWCF_CERES         N1850_f19_tn14_r227_ctrl  310-340  2.981   -45.605   
                                             80-110   2.598   -45.988   
SWCF_CERES-EBAF    N1850_f19_tn14_r227_ctrl  310-340  1.466   -45.605   
                                             80-110   1.083   -45.988   
SWCF_ERBE          N1850_f19_tn14_r227_ctrl  310-340  8.558   -45.605   
                                             80-110   8.175   -45.988   
TREFHT_JRA25       N1850_f19_tn14_r227_ctrl  310-340  1.615   289.220   
                                             80-110   0.431   288.036   
TREFHT_LEGATES     N1850_f19_tn14_r227_ctrl  310-340  1.576   289.220   
                                             80-110   0.392   288.036   
TS_LAND_NCEP       N1850_f19_tn14_r227_ctrl  310-340  3.945   285.100   
                                             80-110   2.181   283.336   
TS_NCEP            N1850_f19_tn14_r227_ctrl  310-340  2.767   290.450   
                                             80-110   1.564   289.247   
U_200_JRA25        N1850_f19_tn14_r227_ctrl  310-

In [45]:
bias_mean.unstack(level=(1,2))

,,Model,Obs,RMSE
,Run,N1850_f19_tn14_r265_ctrl_20180411,N1850_f19_tn14_r265_ctrl_20180411,N1850_f19_tn14_r265_ctrl_20180411
,Years,90-120,90-120,90-120
Variable,Bias,,,
CLDTOT_CLOUDSAT,3.923,70.747,66.824,10.561
CLDTOT_ISCCP,3.947,70.747,66.800,12.485
FLDS_ISCCP,4.507,347.854,343.347,15.278
FLNS_ISCCP,6.925,56.350,49.425,13.988
FLUTC_CERES,-2.009,264.869,266.878,6.708
FLUTC_CERES-EBAF,-1.182,264.869,266.051,4.778
FLUTC_ERBE,0.440,264.869,264.429,4.878
